# <font color='red'><i>Ocean 569A Notes Number 2</i></font>


### The Python code used to produce Figure 2 in Notes No. 2 is given below.


In [ ]:
# code to read the MBARI 19 year T/S dataset and plot the T and S spectra
#
import numpy as np
import matplotlib.pyplot as plt
import random
#
nn=156885
mm=int(nn/2+1)
nsk=6-
eps=0.002
#
# define the input paths
#
path_in='/Users/riser/Desktop/ocean.569A/datasets/MBARI.M1.txt'
#
# begin reading the data using readline
#
data_in=open(path_in,'r')
#
# define some arrays
#
time=np.zeros(nn)
month=np.zeros(nn)
tt=np.zeros(nn)
tt_nomean=np.zeros(nn)
ss=np.zeros(nn)
ss_nomean=np.zeros(nn)
freq=np.zeros(mm)
#
# skip the header stuff
#
for i in range(0,nsk):
    data_read=data_in.readline()
#
# initialize counters and begin reading one line at a time
# strip off the end-of-line character, then split the line into
# the things we want
# edit for bad or missing data; here we'll just do a quick fill
# that is good enough for demonstration purposes
#
count_tt=0
count_ss=0
#
for i in range(0,nn):
   data_read=data_in.readline()
   raw_data0=data_read.strip('\n')
   raw_data1=raw_data0.replace(',',' ')
   rr=raw_data1.split()
   month[i]=float(rr[0][0:2])
   time[i]=i/(24.*365)
   tt[i]=rr[6]
   ss[i]=rr[5]
   if tt[i] > 100:
       tt[i]=tt[77832]
   if ss[i] > 36. or ss[i] < 31.5:
       count_ss+=1
       ss[i]=ss[i-1]*(1.+eps*(random.random()-0.5))
#
data_in.close()
#
# remove the mean from temperature and salinity prior to estimating the spectra
#
tt_nomean=tt-np.mean(tt)
ss_nomean=ss-np.mean(ss)
#
fs=14
#
# define the spectral parameters
#
delt=1./24.
T_length=nn
pi=np.pi
omega0=2.*pi/(T_length*delt)
#
# define the frequency array
#
for i in range(0,mm):
    freq[i]=i*omega0
#
# compute the fft of the input data (temperature in this case)
# the fft will yield a set of complex numbers
# also compute their complex conjugates
# multiply these together to get the spectrum
#
zz_tt=np.fft.rfft(tt_nomean,n=nn)
fourier_amp=np.sqrt((np.real(zz_tt)**2+np.imag(zz_tt)**2))
fourier_phase=180.*np.arctan2(np.imag(zz_tt),np.real(zz_tt))/pi
spec_tt=np.real(zz_tt*np.conj(zz_tt))/(2.*pi*T_length*delt)
spec_tt_amp=(np.absolute(zz_tt))**2/(2.*pi*T_length*delt)
#
# plot the temperature spectrum as a function of frequency
# show the maximum and minimum frequencies
#
fig3=plt.figure(figsize=(7,7))
plt.ylim(0.01,6.e5)
plt.loglog(freq,spec_tt,color='darkkhaki')
plt.xlabel('$\omega$ (radians/day)',fontsize=15,ha='center')
plt.ylabel('Energy Density ($^o$C$^2$)/(radian/day))',fontsize=15)
freq_nyquist=pi/delt
freq_T=2.*pi/(nn*delt)
plt.plot([freq_nyquist,freq_nyquist],[0.01,6.e5],'--k')
plt.text(28.,5.e4,'$\omega_{max}$',fontsize=12,color='blue')
plt.plot([freq_T,freq_T],[0.01,6.e5],'--k',zorder=10)
plt.text(1.2e-3,0.3,'$\omega_o$',fontsize=12,color='blue')
#
# show the the M2 and K1 tide and add a grid
#
plt.text(5.,1.5e2,'K1',color='blue')
plt.text(10.,2.5e1,'M2',color='blue')
plt.text(5./365.,1.7e5,'1 yr',color='blue')
plt.grid()
plt.show()
#
# now do the same for the salinity spectrum
#
zz_ss=np.fft.rfft(ss_nomean,n=nn)
fourier_amp=np.sqrt((np.real(zz_ss)**2+np.imag(zz_ss)**2))
fourier_phase=180.*np.arctan2(np.imag(zz_ss),np.real(zz_ss))/pi
spec_ss=np.real(zz_ss*np.conj(zz_ss))/(2.*pi*T_length*delt)
spec_ss_amp=(np.absolute(zz_ss))**2/(2.*pi*T_length*delt)
#
# plot the salinity spectrum as a function of frequency
# show the maximum and minimum frequencies
#
fig4=plt.figure(figsize=(7,7))
plt.ylim(1.e-4,1.e4)
plt.loglog(freq,spec_ss,color='dodgerblue')
plt.xlabel('$\omega$ (radians/day)',fontsize=15,ha='center')
plt.ylabel('Energy Density (PSU$^2$)/(radian/day))',fontsize=15)
freq_nyquist=pi/delt
freq_T=2.*pi/(nn*delt)
plt.plot([freq_nyquist,freq_nyquist],[1.e-4,1.e4],'--k')
plt.text(28.,5.e3,'$\omega_{max}$',fontsize=12,color='purple')
plt.plot([freq_T,freq_T],[1.e-4,1.e4],'--k',zorder=10)
plt.text(1.2e-3,0.03,'$\omega_o$',fontsize=12,color='purple')
#
# show the the annual peak and add a grid
#
plt.text(5./365.,3.e3,'1 yr',color='purple')
plt.grid()
#plt.savefig(path_out4)
plt.show()
#

### The Python code used to produce Figure 6 in Notes No. 2 is given below.

In [ ]:
# file to read and plot WHOI HOT temperature and salinity cross-spectral ampliude
#
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from netCDF4 import Dataset
#
# read and estimate spectra from WHOI HOT mooring data
#
# define the netcdf reading function
#
def import_data(file_name):   
    '''This function works regardless of the file called as long as a list of 
    variables can be identified easily in the file.'''
    
    data_netcdf = Dataset(file_name, mode = 'r')
    data = {}
#    
    for vname in list(data_netcdf.variables):
        data[str(vname)] = data_netcdf.variables[vname][:] 
#            
    data_netcdf.close()
#            
    return data
#
# define a function to band-average the spectrum to eliminate some noise
#
def band_average(fft_var1,fft_var2,frequency,n_av):
#
# fft_var1 and fft_var2 are the inputs computed via fft
# they can be the same variable or different variables
# n_av is the number of bands to be used for smoothing (nice if it is an odd number)
# this function is limnited to 100,000 points but can easily be modified
#
    nmax=100000
#
# define some variables and arrays
#
    n_spec=len(fft_var1)
    n_av2=int(n_av//2+1)
    spec_amp_av=np.zeros(nmax)
    spec_phase_av=np.zeros(nmax)
    freq_av=np.zeros(nmax)
#
# average the lowest frequency bands first (with half as many points in the average)
# 
    sum_low_amp=0.
    sum_low_phase=0.
    count=0
    spectrum_amp=np.absolute(fft_var1*np.conj(fft_var2))
    spectrum_phase=np.angle(fft_var1*np.conj(fft_var2),deg=True)
#
    for i in range(0,n_av2):
        sum_low_amp+=spectrum_amp[i]
        sum_low_phase+=spectrum_phase[i]
#
    spec_amp_av[0]=sum_low_amp/n_av2
    spec_phase_av[0]=sum_low_phase/n_av
#
# compute the rest of the averages
#
    for i in range(n_av2,n_spec-n_av,n_av):
        count+=1
        spec_amp_est=np.mean(spectrum_amp[i:i+n_av])
        spec_phase_est=np.mean(spectrum_phase[i:i+n_av])
        freq_est=frequency[i+n_av//2]
        spec_amp_av[count]=spec_amp_est
        spec_phase_av[count]=spec_phase_est
        freq_av[count]=freq_est
#
# contract the arrays
#
    spec_amp_av=spec_amp_av[0:count]
    spec_phase_av=spec_phase_av[0:count]
    freq_av=freq_av[0:count]
    return spec_amp_av,spec_phase_av,freq_av,count    
#    
# main program
#
# define the input and output files
#
path_in='/Users/riser/Desktop/ocean.569A/datasets/OS_WHOTS_201606_D_MICROCAT-025m.nc'
path_out='/Users/riser/Desktop/ocean.569A/full_averaged_cross-spectrum.jpg'
#
# read the input file (netcdf)
#
HOT_data=import_data(path_in)
#
# determine the length of the data and the resulting number of spectral estimates
#
nn=len(HOT_data['TIME'])
print ('nn=',nn)
mm=int(nn/2+1)
#
# define the data arrays
#
time_meas=np.zeros(nn)
time_meas_days=np.zeros(nn)
temp_meas=np.zeros(nn)
sal_meas=np.zeros(nn)
tt=np.zeros(nn)
ss=np.zeros(nn)
freq=np.zeros(mm)
#
# parse the time, temperature, and salinity data from the input file

for i in range(0,nn):
   time_meas[i]=24.*(float(HOT_data['TIME'][i])-float(HOT_data['TIME'][0]))
   time_meas_days[i]=(float(HOT_data['TIME'][i])-float(HOT_data['TIME'][0]))
   temp_meas[i]=float(HOT_data['TEMP'][i])
   sal_meas[i]=float(HOT_data['PSAL'][i])
#
# remove the temperature and salinity means from the data
#
tt=temp_meas-np.mean(temp_meas)
ss=sal_meas-np.mean(sal_meas)
# 
# determine the frequencies for the spectrum
#
delt=0.00208333
T_length=nn
pi=np.pi
omega0=2.*pi/(T_length*delt)
#
for i in range(0,mm):
    freq[i]=i*omega0
#
# compute the fft of the input data (temperature in this case)
# the fft will yield a set of complex numbers
# also compute their complex conjugates
# multiply these together to get the spectrum
#
n_av=21
zz1=np.fft.rfft(tt,n=nn)
zz2=np.fft.rfft(ss,n=nn)
zz2_star=np.conj(zz2)
cospec_amp,cospec_phase,freq_av,count=band_average(zz1,zz2,freq,n_av)
#
# begin plotting the spectrum
#
fig1=plt.figure(figsize=(7,7))
#
plt.ylim(1.,3.e7)
plt.loglog(freq_av,cospec_amp,color='purple')
plt.xlabel('$\omega$ (radians/day)',fontsize=15,ha='center')
plt.ylabel(' ($(^o$C)(PSU)/RPD)',fontsize=15)
freq_nyquist=pi/delt
freq_T=2.*pi/(nn*delt)
plt.plot([freq_nyquist,freq_nyquist],[1.,3.e7],'--k')
plt.text(7.e2,5.e4,'$\omega_{max}$',fontsize=12,color='firebrick')
plt.text(1.5e2,1.6e6,'n_av = 21')
plt.grid(which='both')
plt.savefig(path_out)
plt.show()
#


### The Python code used to produce Figure 7 in Notes No. 2 is given below.

In [ ]:
# file to read HOT T and S data from WHOTS mooring and to
# estimate coherence and phase between T and S
#
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from netCDF4 import Dataset
#
# read and plot WHOI HOT mooring data
#
# define the netcdf reading function
#
def import_data(file_name):   
    '''This function works regardless of the file called as long as a list of 
    variables can be identified easily in the file.'''
    
    data_netcdf = Dataset(file_name, mode = 'r')
    data = {}
#    
    for vname in list(data_netcdf.variables):
        data[str(vname)] = data_netcdf.variables[vname][:] 
#            
    data_netcdf.close()
#            
    return data
#
# define a function to band-average the spectrum to eliminate some noise
#
def band_average(fft_var1,fft_var2,frequency,n_av):
#
# this is a function to estimate the autospectrum or co-spectrum for 2 variables
# fft_var1 and fft_var2 are the inputs computed via fft
# they can be the same variable or different variables
# n_av is the number of bands to be used for smoothing (nice if it is an odd number)
# this function is limnited to 100,000 points but can easily be modified
#
    nmax=100000
#
# define some variables and arrays
#
    n_spec=len(fft_var1)
    n_av2=int(n_av//2+1)
    spec_amp_av=np.zeros(nmax)
    spec_phase_av=np.zeros(nmax)
    freq_av=np.zeros(nmax)
#
# average the lowest frequency bands first (with half as many points in the average)
# 
    sum_low_amp=0.
    sum_low_phase=0.
    count=0
    spectrum_amp=np.absolute(fft_var1*np.conj(fft_var2))
    spectrum_phase=np.angle(fft_var1*np.conj(fft_var2),deg=True)
#
    for i in range(0,n_av2):
        sum_low_amp+=spectrum_amp[i]
        sum_low_phase+=spectrum_phase[i]
#
    spec_amp_av[0]=sum_low_amp/n_av2
    spec_phase_av[0]=sum_low_phase/n_av
#
# compute the rest of the averages
#
    for i in range(n_av2,n_spec-n_av,n_av):
        count+=1
        spec_amp_est=np.mean(spectrum_amp[i:i+n_av])
        spec_phase_est=np.mean(spectrum_phase[i:i+n_av])
        freq_est=frequency[i+n_av//2]
        spec_amp_av[count]=spec_amp_est
        spec_phase_av[count]=spec_phase_est
        freq_av[count]=freq_est
#
# contract the arrays
#
    spec_amp_av=spec_amp_av[0:count]
    spec_phase_av=spec_phase_av[0:count]
    freq_av=freq_av[0:count]
    return spec_amp_av,spec_phase_av,freq_av,count    
#    
# main program
#
# define the input and output files
#
path_in='/Users/riser/Desktop/ocean.569A/datasets/OS_WHOTS_201606_D_MICROCAT-025m.nc'
path_out1='/Users/riser/Desktop/ocean.569A/HOT.coherence.jpg'
path_out2='/Users/riser/Desktop/ocean.569A/HOT.phase.jpg'
#
# read the input file (netcdf)
#
HOT_data=import_data(path_in)
#
# determine the length of the data and the resulting number of spectral estimates
#
nn=len(HOT_data['TIME'])
print ('nn=',nn)
mm=int(nn/2+1)
#
# define the data arrays
#
time_meas=np.zeros(nn)
time_meas_days=np.zeros(nn)
temp_meas=np.zeros(nn)
sal_meas=np.zeros(nn)
tt=np.zeros(nn)
ss=np.zeros(nn)
freq=np.zeros(mm)
#
# parse the time, temperature, and salinity data from the input file

for i in range(0,nn):
   time_meas[i]=24.*(float(HOT_data['TIME'][i])-float(HOT_data['TIME'][0]))
   time_meas_days[i]=(float(HOT_data['TIME'][i])-float(HOT_data['TIME'][0]))
   temp_meas[i]=float(HOT_data['TEMP'][i])
   sal_meas[i]=float(HOT_data['PSAL'][i])
#
# remove the temperature and salinity means from the data
#
tt=temp_meas-np.mean(temp_meas)
ss=sal_meas-np.mean(sal_meas)
# 
# determine the frequencies for the spectrum
#
delt=0.00208333
T_length=nn
pi=np.pi
omega0=2.*pi/(T_length*delt)
#
for i in range(0,mm):
    freq[i]=i*omega0
#
# compute the fft of the input data (temperature and salinity here)
# the fft will yield a set of complex numbers
# also compute their complex conjugates
# use the function band_average to get the spectral estimates
# estimate the coherence and phase using the results of the function
#
n_av=21
zz1=np.fft.rfft(tt,n=nn)
zz2=np.fft.rfft(ss,n=nn)
zz2_star=np.conj(zz2)
temp_spec,temp_phase,freq_av,count=band_average(zz1,zz1,freq,n_av)
salt_spec,salt_phase,freq_av,count=band_average(zz2,zz2,freq,n_av)
cospec_amp,cospec_phase,freq_av,count=band_average(zz1,zz2_star,freq,n_av)
coh_sq=cospec_amp**2/(temp_spec*salt_spec)
#
# begin plotting the coherence and phase
#
# first the coherence
#
fig1=plt.figure(figsize=(9,7))
plt.ylim(0.,1.)
plt.semilogx(freq_av,coh_sq,color='purple')
plt.xlabel('$\omega$ (radians/day)',fontsize=15,ha='center')
plt.ylabel('Squared Coherence $\it{T}$-$\it{S}$',fontsize=15)
freq_nyquist=pi/delt
freq_T=2.*pi/(nn*delt)
plt.plot([freq_nyquist,freq_nyquist],[0.,1.],'--k')
plt.text(8.e2,0.2,'$\omega_{max}$',fontsize=12,color='firebrick')
plt.text(1.2,0.88,'n_av = 21')
plt.grid(which='both')
plt.title('MBARI M1 Mooring')
plt.show()
plt.savefig(path_out1)
plt.show()
#
# now the phase
#
fig2=plt.figure(figsize=(9,7))
plt.ylim(-180.,180.)
plt.semilogx(freq_av,cospec_phase,color='orange')
plt.xlabel('$\omega$ (radians/day)',fontsize=15,ha='center')
plt.ylabel('Phase $\it{T}$-$\it{S}$, degrees',fontsize=15)
freq_nyquist=pi/delt
freq_T=2.*pi/(nn*delt)
plt.plot([freq_nyquist,freq_nyquist],[-180.,180.],'--k')
plt.text(8.e2,-110.,'$\omega_{max}$',fontsize=12,color='firebrick')
plt.text(1.2,130.,'n_av = 21')
plt.grid(which='both')
plt.title('MBARI M1 Mooring')
plt.savefig(path_out2)
plt.show()